In [119]:
import pandas as pd
import seaborn as sns

In [120]:
df = pd.read_csv('solicitacoescredito.csv')

In [121]:
df.head()

,numero_solicitacao,razaoSocial,nomeFantasia,cnpjSemTraco,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,scorePontualidade,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista
0,1,James Richardson-Patel,Alexandra Williams,KEBE17609492220843,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,1.0,43200.0,2020-02-03T20:57:33
1,2,Dr. Geoffrey Walsh,Mr. Darren Arnold,JRBK88908250677300,0,0.000000,NaN,2019-06-12T17:28:31,0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,3,Joanna Hudson,Dr. David Rees,GCVQ28531614261293,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,...,7.0,0.0,2014.0,De 6 a 10 anos,20000.0,False,True,1.0,4320.0,2020-02-04T16:40:49
3,4,Gordon Jones-Hopkins,Sara Reid-Robson,KJND32266018316396,20,0.000000,NaN,2017-02-13T17:20:27,0,0.00,...,12.0,0.0,2013.0,De 6 a 10 anos,30000.0,False,True,0.0,5920.0,2020-02-04T16:37:52
4,5,Nigel Lee,Dr. Stanley Duncan,CGQN15826802440348,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,1.0,89000.0,2020-02-04T15:06:28


# Verificando se existem dados nulos

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8973 entries, 0 to 8972
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   numero_solicitacao           8973 non-null   int64  
 1   razaoSocial                  8973 non-null   object 
 2   nomeFantasia                 8973 non-null   object 
 3   cnpjSemTraco                 8973 non-null   object 
 4   maiorAtraso                  8973 non-null   int64  
 5   margemBrutaAcumulada         8973 non-null   float64
 6   percentualProtestos          7475 non-null   float64
 7   primeiraCompra               8867 non-null   object 
 8   prazoMedioRecebimentoVendas  8973 non-null   int64  
 9   titulosEmAberto              8973 non-null   float64
 10  valorSolicitado              8973 non-null   float64
 11  status                       8973 non-null   object 
 12  definicaoRisco               8973 non-null   object 
 13  diferencaPercentua

## É possível analisar que:
 - existem muitos dados faltantes
 - dentre as 38 features desse dataset, existe uma variedade de dados que retornam números e dados que retornam objetos (strings) e muitos dados que são inuteis como NaN ou valor zerados que terão que ser limpos

In [123]:
df.describe()

,numero_solicitacao,maiorAtraso,margemBrutaAcumulada,percentualProtestos,prazoMedioRecebimentoVendas,titulosEmAberto,valorSolicitado,diferencaPercentualRisco,percentualRisco,dashboardCorrelacao,...,duplicatasAReceber,estoque,faturamentoBruto,margemBruta,periodoDemonstrativoEmMeses,custos,anoFundacao,capitalSocial,scorePontualidade,limiteEmpresaAnaliseCredito
count,8973.000000,8973.000000,8973.000000,7475.000000,8973.000000,8.973000e+03,8.973000e+03,8973.000000,8973.000000,8973.000000,...,4.733000e+03,4.733000e+03,8.223000e+03,8.223000e+03,8223.000000,8.223000e+03,8228.000000,8.228000e+03,8973.000000,8.228000e+03
mean,4550.042015,24.642594,0.362176,0.019260,23.083027,6.487101e+04,7.492436e+05,0.750321,0.249679,0.047236,...,1.663397e+07,1.523956e+07,5.597420e+07,1.620988e+07,10.377356,2.839061e+07,2006.027467,1.121453e+07,0.798346,2.851017e+06
std,2603.485853,66.180793,0.201455,0.593579,68.177649,2.482852e+05,2.261875e+07,0.146058,0.146058,0.472476,...,2.979029e+08,8.383792e+07,3.344357e+08,1.163482e+08,3.220965,2.072148e+08,19.424120,9.742853e+07,0.379186,2.672324e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000e+02,0.207547,0.000000,-0.999990,...,-2.278071e+07,-2.632260e+05,0.000000e+00,-6.148721e+08,1.000000,-3.466338e+08,1000.000000,0.000000e+00,0.000000,0.000000e+00
25%,2316.000000,3.000000,0.281395,0.000000,0.000000,0.000000e+00,2.500000e+04,0.642857,0.142857,0.000000,...,3.920500e+04,1.712860e+05,1.191995e+06,0.000000e+00,11.000000,0.000000e+00,2000.000000,5.000000e+04,0.887479,7.360000e+03
50%,4559.000000,6.000000,0.402895,0.000000,0.000000,0.000000e+00,5.000000e+04,0.750000,0.250000,0.000000,...,1.088164e+06,1.063783e+06,3.599483e+06,0.000000e+00,12.000000,0.000000e+00,2009.000000,1.000000e+05,1.000000,4.860000e+04
75%,6802.000000,22.000000,0.507860,0.000000,30.000000,1.725000e+04,1.200000e+05,0.857143,0.357143,0.000000,...,6.576243e+06,5.493839e+06,1.584222e+07,3.357474e+06,12.000000,4.520907e+06,2015.000000,5.000000e+05,1.000000,3.450000e+05
max,9045.000000,1265.000000,1.000000,36.983728,1605.000000,3.938590e+06,1.500000e+09,1.000000,0.792453,0.999990,...,2.009358e+10,1.293428e+09,6.426115e+09,3.366843e+09,12.000000,4.393536e+09,2020.000000,4.100000e+09,1.000000,1.974261e+09


# Escolhas feitas

Após uma analise do desafio, colunas e os dados presentes nelas, decidimos dividir em dois projetos:
1. Um agrupamento de empresas para para que podemos oferecer as melhores descontos, encerrar contratos ou apenas deixar na neutralidade
2. Previsão de demanda futura com precisão para otimizar o armazenamento do estoque da ABX tenologia

# Agrupamento

## Remoção das colunas desnecessarias

In [124]:
colunas_necessarias=[
    "maiorAtraso",
    "primeiraCompra" ,
    "margemBrutaAcumulada",
    "percentualProtestos",
    "prazoMedioRecebimentoVendas",
    "titulosEmAberto",
    "status",
    "definicaoRisco",
    "passivoCirculante",
    "endividamento",
    "faturamentoBruto",
    "restricoes",
    "scorePontualidade"
]

In [125]:
df = df[colunas_necessarias]

In [126]:
df = df.dropna()
df.head()

,maiorAtraso,primeiraCompra,margemBrutaAcumulada,percentualProtestos,prazoMedioRecebimentoVendas,titulosEmAberto,status,definicaoRisco,passivoCirculante,endividamento,faturamentoBruto,restricoes,scorePontualidade
4,20,2010-07-13T00:00:00,0.454088,0.0,20,1486.95,AprovadoAnalista,De 11 a 30 % - Baixo,12149031.0,3039112.0,40779757.0,False,1.000000
7,4,2007-07-02T00:00:00,0.350175,0.0,102,208980.00,AprovadoAnalista,De 0 a 10 % - Muito Baixo,2538069.0,0.0,11958227.0,False,1.000000
8,43,2006-07-10T00:00:00,0.245818,0.0,27,6432.00,AprovadoAnalista,De 11 a 30 % - Baixo,13843198.0,0.0,72479551.0,False,1.000000
14,4,2019-03-08T00:00:00,0.314575,0.0,0,0.00,AprovadoAnalista,De 11 a 30 % - Baixo,11680162.0,1567918.0,36783195.0,False,1.000000
19,5,2016-01-20T00:00:00,0.394118,0.0,0,0.00,AprovadoAnalista,De 11 a 30 % - Baixo,50773.0,0.0,57950454.0,False,0.999976


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3929 entries, 4 to 8963
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   maiorAtraso                  3929 non-null   int64  
 1   primeiraCompra               3929 non-null   object 
 2   margemBrutaAcumulada         3929 non-null   float64
 3   percentualProtestos          3929 non-null   float64
 4   prazoMedioRecebimentoVendas  3929 non-null   int64  
 5   titulosEmAberto              3929 non-null   float64
 6   status                       3929 non-null   object 
 7   definicaoRisco               3929 non-null   object 
 8   passivoCirculante            3929 non-null   float64
 9   endividamento                3929 non-null   float64
 10  faturamentoBruto             3929 non-null   float64
 11  restricoes                   3929 non-null   object 
 12  scorePontualidade            3929 non-null   float64
dtypes: float64(7), int64(2)

## Transformando a coluna de primeira compra

In [128]:
import datetime
import math
TODAY = datetime.date.today()
THRESHOLD = 5

def calculate_fidelity(first_buy):
    date, _ = first_buy.split('T')
    year, month, day = date.split('-')
    actual_date = datetime.date(int(year), int(month), int(day))

    years = (TODAY - actual_date).days / 365

    return math.floor(years / THRESHOLD)


In [129]:
df_copy = df

In [130]:
df_copy['primeiraCompra'] = df_copy['primeiraCompra'].apply(calculate_fidelity)


In [131]:
df_copy.head()

,maiorAtraso,primeiraCompra,margemBrutaAcumulada,percentualProtestos,prazoMedioRecebimentoVendas,titulosEmAberto,status,definicaoRisco,passivoCirculante,endividamento,faturamentoBruto,restricoes,scorePontualidade
4,20,2,0.454088,0.0,20,1486.95,AprovadoAnalista,De 11 a 30 % - Baixo,12149031.0,3039112.0,40779757.0,False,1.000000
7,4,3,0.350175,0.0,102,208980.00,AprovadoAnalista,De 0 a 10 % - Muito Baixo,2538069.0,0.0,11958227.0,False,1.000000
8,43,3,0.245818,0.0,27,6432.00,AprovadoAnalista,De 11 a 30 % - Baixo,13843198.0,0.0,72479551.0,False,1.000000
14,4,1,0.314575,0.0,0,0.00,AprovadoAnalista,De 11 a 30 % - Baixo,11680162.0,1567918.0,36783195.0,False,1.000000
19,5,1,0.394118,0.0,0,0.00,AprovadoAnalista,De 11 a 30 % - Baixo,50773.0,0.0,57950454.0,False,0.999976


## Amostragem de dados em gráficos

# Pré-processamento de dados

## Normalização de dados

 A normalização refere-se ao processo de ajustar a escala dos valores numéricos em um conjunto de dados para uma escala comum, geralmente entre 0 e 1 ou com uma média de zero e um desvio padrão de um.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler(feature_range=(0,1))

scaled_standard = scaler_standard.fit_transform(df[[]])
#passando colunas específicas minmax
scaled_minmax = scaler_minmax.fit_transform(df[[]])
# passando dataframe todo minmax
scaled_df = scaler_minmax.fit_transform()

print(scaled_standard)
print(scaled_minmax)

## Codificação de atributos categóricos

A codificação de atributos categóricos refere-se ao processo de converter atributos categóricos em uma forma numérica que pode ser utilizada por algoritmos de aprendizado de máquina.

In [ ]:
# O Label Encoding é utilizado para converter valores categóricos em números inteiros.
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

# ou

#O One-Hot Encoding é utilizado para converter valores categóricos em vetores binários.
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()